<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Connection" data-toc-modified-id="Connection-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Connection</a></span></li><li><span><a href="#Load-Data" data-toc-modified-id="Load-Data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Load Data</a></span></li><li><span><a href="#Split-training/test-dataset-and-select-categories/store" data-toc-modified-id="Split-training/test-dataset-and-select-categories/store-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Split training/test dataset and select categories/store</a></span></li><li><span><a href="#Model:-PRICE_TO_MINUS_ELASTICITY-as-regressor" data-toc-modified-id="Model:-PRICE_TO_MINUS_ELASTICITY-as-regressor-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Model: PRICE_TO_MINUS_ELASTICITY as regressor</a></span></li><li><span><a href="#RMSE-Evaluation" data-toc-modified-id="RMSE-Evaluation-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>RMSE Evaluation</a></span><ul class="toc-item"><li><span><a href="#Product-Level" data-toc-modified-id="Product-Level-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Product Level</a></span></li><li><span><a href="#Sub-Category-Level" data-toc-modified-id="Sub-Category-Level-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Sub-Category Level</a></span></li><li><span><a href="#Category-Level" data-toc-modified-id="Category-Level-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>Category Level</a></span></li></ul></li></ul></div>

In [ ]:
import configparser
import snowflake.connector
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf 
from statsmodels.tsa.seasonal import seasonal_decompose 
from sklearn.metrics import mean_squared_error
from statsmodels.tools.eval_measures import rmse
from math import sqrt
from statsmodels.tsa.stattools import adfuller
from fbprophet import Prophet
from fbprophet.diagnostics import cross_validation
from tqdm import tqdm
from tqdm._tqdm import trange
from sklearn.metrics import r2_score
import datetime
from sklearn.preprocessing import LabelEncoder
import glob
import os
import warnings
warnings.filterwarnings("ignore")

# Connection

In [ ]:
config_file = 'credentials.conf'

In [ ]:
credentials = configparser.ConfigParser()
credentials.read(config_file)
connection = snowflake.connector.connect(**credentials[''])          
cursor = connection.cursor()
cursor.execute("USE DATABASE {database}".format(**credentials['']))
cursor.execute("USE SCHEMA {schema}".format(**credentials['']))

# Load Data

In [ ]:
sales = pd.read_sql('''
                       SELECT *
                       FROM SALES_HISTORY_05282020
                    ''', connection)

In [ ]:
elasticity = pd.read_sql('''
                       SELECT *
                       FROM ELASTICITY_04212020
                    ''', connection)

In [ ]:
elasticity.head()

,PRODUCT_ID,ELASTICITY
0,0001,0.64
1,0002,0.46
2,0003,1.54
3,0004,0.21
4,0005,0.69


In [ ]:
sales_copy = sales.copy()

In [ ]:
elasticity_copy = elasticity.copy()

In [ ]:
sales_copy = pd.merge(sales_copy, elasticity_copy, how='left', on=['PRODUCT_ID'])

In [ ]:
sales_copy["PRICE_TO_MINUS_ELASTICITY"] = np.power(sales_copy["SALES_PRICE"],(-sales_copy["ELASTICITY"]))

In [ ]:
# transform SALES_TYPE_ID to numerical
le = LabelEncoder()
sales_copy["SALES_TYPE_ID"] = le.fit_transform(sales_copy["SALES_TYPE_ID"])

# Split training/test dataset and select categories/store

The test period is from September 07, 2019 to December 28, 2019.  
The training period is prior to the test, i.e. September 02, 2017 to August 31, 2019
  
We select category 24, 18, 04  
We select store 16

In [ ]:
TRAIN_START = "2017-09-02"
TRAIN_END = "2019-08-31"
TEST_START = "2019-09-07"
TEST_END = "2019-12-28"
TEST_END_NEXT_WEEK = "2020-01-04"
TEST_PERIOD = (pd.to_datetime(TEST_END)-pd.to_datetime(TEST_START))/np.timedelta64(1,'W')+1

selected_category = ['24','18','04']
selected_store = ['16']

# Model: PRICE_TO_MINUS_ELASTICITY as regressor

Parameter: yearly_seasonality=True, interval_width=0.95, n_changepoints = 25, changepoint_prior_scale = 1  
Regressor: PRICE_TO_MINUS_ELASTICITY

In [ ]:
# create result dataframe
Prophet_fit_product_level = pd.DataFrame(columns = ["CATEGORY", "SUB_CATEGORY", "STORE_ID", "PRODUCT_ID", "rmse_E3", "rmse_prophet", "quantity", "e3", "yhat"])

In [ ]:
# create the path to store plots and summary csv files
path = './Prophet_figs_product_level/'
os.mkdir(path)
os.mkdir(path+'Prophet_better_plots')
os.mkdir(path+'E3_better_plots')

In [ ]:
def modeling(category, store, subcat, product): 
    # prepare data
    specific_sales = sales_copy[(sales_copy['CATEGORY']==category) & (sales_copy['STORE_ID']==store) &(sales_copy["SUB_CATEGORY"] == subcat) &(sales_copy["PRODUCT_ID"] == product)]
    sum_sales = specific_sales.groupby("WEEKLY_SALES_DATE")["QUANTITY", "PRICE_TO_MINUS_ELASTICITY"].agg({"QUANTITY":'sum', "PRICE_TO_MINUS_ELASTICITY":'mean'})
    idx = pd.date_range(start = specific_sales["WEEKLY_SALES_DATE"].min(),
                        end   = TEST_END_NEXT_WEEK, freq = "W").shift(-1, freq = 'D')
    E3_predicted_units = specific_sales.groupby("WEEKLY_SALES_DATE")['PREDICTED_UNITS'].sum()
    E3_predicted_units.index = E3_predicted_units.index.astype("datetime64[ns]")
    sum_sales.index = sum_sales.index.astype("datetime64[ns]")
    sum_sales = sum_sales.reindex(idx, fill_value = 0) # fill missing value with zero
    sum_sales_train = sum_sales.loc[TRAIN_START:TRAIN_END]
    sum_sales_train = sum_sales_train.reset_index().rename(columns = {"index":'ds', "QUANTITY": "y"})
    sum_sales_test = sum_sales.loc[TEST_START:TEST_END]

    # train model
    prophet_model = Prophet(yearly_seasonality=True, interval_width=0.95, n_changepoints = 25, changepoint_prior_scale = 1, weekly_seasonality=False,daily_seasonality=False)
    prophet_model.add_regressor('PRICE_TO_MINUS_ELASTICITY')
    prophet_model.fit(sum_sales_train)
    future = prophet_model.make_future_dataframe(periods = int(TEST_PERIOD), freq = "7d")
    future["PRICE_TO_MINUS_ELASTICITY"] = sum_sales["PRICE_TO_MINUS_ELASTICITY"].reset_index(drop = True)

    # prediction
    prediction = prophet_model.predict(future)
    prediction_oos = prediction[(prediction["ds"]>=TEST_START)&(prediction["ds"]<=TEST_END)] 

    # goodness of fit (oos)
    rmse_E3 = sqrt(mean_squared_error(sum_sales_test["QUANTITY"],E3_predicted_units[TEST_START:TEST_END]))
    rmse_prophet = sqrt(mean_squared_error(sum_sales_test["QUANTITY"],prediction_oos["yhat"]))

    # output
    quantity = sum_sales_test["QUANTITY"]
    e3 = E3_predicted_units[TEST_START:TEST_END]
    yhat = prediction_oos["yhat"].values
    performance = {"CATEGORY":category, "SUB_CATEGORY":subcat, "STORE_ID":store, "PRODUCT_ID":product, "quantity":quantity, "e3":e3, "yhat":yhat, \
                  "rmse_E3":rmse_E3, "rmse_prophet":rmse_prophet}
    performance = pd.DataFrame(performance, index = sum_sales_test.index)
    
    # plot
    prophet_model.plot(prediction)
    E3_predicted_units.plot(color = 'r')
    plt.plot(sum_sales_test["QUANTITY"], color = 'purple', linestyle = ":", linewidth = 3)
    plt.title("Engage3 V.S Prophet: PRODUCT ID: "+product+", category: "+category+", rmse_E3="+str(round(rmse_E3,2))+", rmse_prophet="+str(round(rmse_prophet, 2)) )
    if rmse_E3 < rmse_prophet:
        plt.savefig(path+'E3_better_plots/cat_'+str(category)+'_st_'+str(store)+'_pro_'+str(product)+'.png', bbox_inches = 'tight')
    else:
        plt.savefig(path+'Prophet_better_plots/cat_'+str(category)+'_st_'+str(store)+'_pro_'+str(product)+'.png', bbox_inches = 'tight')
    plt.clf()
    
    return performance

In [ ]:
for category in tqdm(selected_category):
    for store in selected_store:
        selected_subcat = sales_copy[(sales_copy['CATEGORY']==category) & (sales_copy['STORE_ID']==store)]['SUB_CATEGORY'].unique()

        for subcat in selected_subcat:
            selected_product = sales_copy[(sales_copy['CATEGORY']==category) & (sales_copy['STORE_ID']==store) &  (sales_copy['SUB_CATEGORY']==subcat)]['PRODUCT_ID'].unique()
            
            for product in selected_product:
                try:
                    # modeling
                    performance = modeling(category, store, subcat, product)
                    Prophet_fit_product_level = Prophet_fit_product_level.append(performance)
                     
                except Exception as e:
                    print(e, "CATEGORY:", category," STORE:", store, " PRODUCT:", product)

    print("CATEGORY: ", category," is done.")

  0%|                                                                                            | 0/3 [00:00<?, ?it/s]

Found input variables with inconsistent numbers of samples: [17, 12] CATEGORY: 24  STORE: 16  PRODUCT: 1139
Found input variables with inconsistent numbers of samples: [17, 16] CATEGORY: 24  STORE: 16  PRODUCT: 0017
Found input variables with inconsistent numbers of samples: [17, 16] CATEGORY: 24  STORE: 16  PRODUCT: 0035
Found input variables with inconsistent numbers of samples: [17, 16] CATEGORY: 24  STORE: 16  PRODUCT: 0951


INFO:fbprophet:n_changepoints greater than number of observations. Using 15.


Dataframe has less than 2 non-NaN rows. CATEGORY: 24  STORE: 16  PRODUCT: 1108
Found input variables with inconsistent numbers of samples: [17, 15] CATEGORY: 24  STORE: 16  PRODUCT: 0103
Found input variables with inconsistent numbers of samples: [17, 11] CATEGORY: 24  STORE: 16  PRODUCT: 0106
Found input variables with inconsistent numbers of samples: [17, 14] CATEGORY: 24  STORE: 16  PRODUCT: 0524
CATEGORY:  24  is done.


 33%|███████████████████████████▋                                                       | 1/3 [06:19<12:39, 379.68s/it]

...

<Figure size 720x432 with 0 Axes>

<Figure size 720x432 with 0 Axes>

<Figure size 720x432 with 0 Axes>

<Figure size 720x432 with 0 Axes>

<Figure size 720x432 with 0 Axes>

In [ ]:
Prophet_fit_product_level.to_pickle("Prophet_fit_product_level.pkl")
Prophet_fit_product_level.to_csv("Prophet_fit_product_level.csv")

# RMSE Evaluation

## Product Level

In [ ]:
Prophet_fit_product_level = pd.read_pickle("Prophet_fit_product_level.pkl")
np.mean(Prophet_fit_product_level[["rmse_E3","rmse_prophet"]])

rmse_E3         112.426232
rmse_prophet    158.803677
dtype: float64

## Sub-Category Level

In [ ]:
model_fit_subcat = pd.read_pickle("Prophet_fit_product_level.pkl")
model_fit_subcat["date"] = model_fit_subcat.index

In [ ]:
model_fit_subcat_2  = pd.DataFrame()

for sub_category in model_fit_subcat['SUB_CATEGORY'].unique():
    data = model_fit_subcat[model_fit_subcat['SUB_CATEGORY'] == sub_category]
    rmse_E3 = sqrt(mean_squared_error(data.groupby('date')['quantity'].sum(), \
                                      data.groupby('date')['e3'].sum()))
    rmse_PROPHET = sqrt(mean_squared_error(data.groupby('date')['quantity'].sum(), \
                                           data.groupby('date')['yhat'].sum()))   
    rmse = {"SUB_CATEGORY":sub_category, "rmse_E3":rmse_E3, "rmse_PROPHET":rmse_PROPHET}
    model_fit_subcat_2 = model_fit_subcat_2.append(rmse, ignore_index=True)

In [ ]:
model_fit_subcat_2.head()

,SUB_CATEGORY,rmse_E3,rmse_PROPHET
0,192,679.460720,304.437147
1,191,608.794594,150.062838
2,098,193.390873,65.305683
3,403,211.347974,36.994626
4,351,87.824022,29.542358


In [ ]:
np.mean(model_fit_subcat_2[["rmse_E3","rmse_PROPHET"]])

rmse_E3         192.426178
rmse_PROPHET    219.644512
dtype: float64

## Category Level

In [ ]:
model_fit_cat = pd.read_pickle("Prophet_fit_product_level.pkl")
model_fit_cat["date"] = model_fit_cat.index

In [ ]:
model_fit_cat_2  = pd.DataFrame()

for category in model_fit_cat['CATEGORY'].unique():
    data = model_fit_cat[model_fit_cat['CATEGORY'] == category]
    rmse_E3 = sqrt(mean_squared_error(data.groupby('date')['quantity'].sum(), \
                                      data.groupby('date')['e3'].sum()))
    rmse_PROPHET = sqrt(mean_squared_error(data.groupby('date')['quantity'].sum(), \
                                           data.groupby('date')['yhat'].sum()))   
    rmse = {"CATEGORY":category, "rmse_E3":rmse_E3, "rmse_PROPHET":rmse_PROPHET}
    model_fit_cat_2 = model_fit_cat_2.append(rmse, ignore_index=True)

In [ ]:
model_fit_cat_2.head()

,CATEGORY,rmse_E3,rmse_PROPHET
0,24,4584.738490,5093.257326
1,18,635.686573,1147.099940
2,04,686.455630,2812.665857


In [ ]:
np.mean(model_fit_cat_2[["rmse_E3","rmse_PROPHET"]])

rmse_E3         1968.960231
rmse_PROPHET    3017.674374
dtype: float64